# Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/employee-turnover-balanced.csv')
df.head()

,left_company,age,frequency_of_travel,department,commuting_distance,education,satisfaction_with_environment,gender,seniority_level,position,satisfaction_with_job,married_or_single,last_raise_pct,last_performance_rating,total_years_working,years_at_company,years_in_current_job,years_since_last_promotion,years_with_current_supervisor
0,No,37,Travel_Rarely,Sales,16,4,4,Male,2,Sales Executive,3,Divorced,19,3,9,1,0,0,0
1,No,39,Travel_Rarely,Research & Development,3,2,3,Male,2,Laboratory Technician,3,Divorced,15,3,11,10,8,0,7
2,No,52,Travel_Frequently,Research & Development,25,4,3,Female,4,Manufacturing Director,4,Married,22,4,31,9,8,0,0
3,No,50,Non-Travel,Sales,1,3,4,Female,2,Sales Executive,3,Married,12,3,19,18,7,0,13
4,No,44,Travel_Rarely,Research & Development,4,3,4,Male,2,Healthcare Representative,2,Single,12,3,10,5,2,2,3


In [2]:
# numerical_vars = []
num_vars = df.select_dtypes(include='int64')

# categorical_vars = []
cat_vars =df.select_dtypes(include='object')

In [3]:
# insert code here
from sklearn.model_selection import train_test_split
X = df.drop('left_company', axis=1)
y = df['left_company']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=124)

print(f'Training samples: {X_train.shape[0]:,}')
print(f'Test samples: {X_test.shape[0]:,}')

Training samples: 800
Test samples: 200


In [4]:
X_train

,age,frequency_of_travel,department,commuting_distance,education,satisfaction_with_environment,gender,seniority_level,position,satisfaction_with_job,married_or_single,last_raise_pct,last_performance_rating,total_years_working,years_at_company,years_in_current_job,years_since_last_promotion,years_with_current_supervisor
294,35,Travel_Rarely,Research & Development,7,3,3,Male,3,Manufacturing Director,3,Single,21,4,17,8,5,1,6
766,33,Travel_Rarely,Research & Development,14,3,3,Male,1,Laboratory Technician,4,Married,13,3,8,5,4,0,4
28,47,Non-Travel,Research & Development,14,4,3,Male,2,Research Scientist,2,Married,18,3,16,8,7,1,7
85,30,Travel_Rarely,Sales,15,2,3,Male,3,Sales Executive,1,Divorced,11,3,12,7,7,1,7
176,27,Travel_Rarely,Research & Development,16,4,3,Female,1,Laboratory Technician,2,Married,14,3,4,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,24,Travel_Rarely,Sales,3,2,1,Female,2,Sales Executive,3,Single,14,3,4,2,2,2,0
135,38,Travel_Rarely,Research & Development,2,5,4,Male,1,Research Scientist,3,Married,13,3,6,1,0,0,1
17,40,Travel_Rarely,Research & Development,10,4,4,Female,1,Laboratory Technician,3,Married,13,3,10,7,7,1,7
668,34,Non-Travel,Research & Development,16,4,4,Male,1,Research Scientist,1,Married,23,4,5,5,2,3,0


In [5]:
y_train

294     No
766    Yes
28      No
85      No
176     No
      ... 
962    Yes
135     No
17      No
668    Yes
462     No
Name: left_company, Length: 800, dtype: object

In [6]:
# insert code here
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression


polys = []
non_polys = num_vars.columns
ohes = cat_vars.columns.drop('left_company')

poly_pipeline = Pipeline([
    ('add_polynomials', PolynomialFeatures(2, include_bias=False)),
    ('standardize_poly', StandardScaler())]
)
    
processing_pipeline = ColumnTransformer(transformers=[
    ('poly_processing', poly_pipeline, polys),
    ('nonpoly_scaling', StandardScaler(), non_polys),
    ('dummys', OneHotEncoder(drop='first'), ohes)]
)

In [7]:
# insert code here
from sklearn.linear_model import LinearRegression

modeling_pipeline = Pipeline([('data_processing', processing_pipeline), 
                              ('logreg', LogisticRegression(penalty=None, max_iter=500))
                              ])

In [8]:
modeling_pipeline.fit(X_train,y_train)
print("Training dataset score is" ,modeling_pipeline.score(X_train, y_train))
print("Testing dataset score is" ,modeling_pipeline.score(X_test, y_test))

Training dataset score is 0.7275
Testing dataset score is 0.67


In [12]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


# Define the hyperparameters to tune and the range of values to search over
param_grid = {
    'logreg__penalty': ['l2'],
    'logreg__C': [0.01, 0.1, 1, 10, 100],
}



# Choose a model to train
model = LogisticRegression(solver='liblinear')


# Create a grid search object
grid_search = GridSearchCV(modeling_pipeline, param_grid=param_grid, cv=5,scoring='accuracy', error_score='raise')

# Train the model using grid search
grid_search.fit(X_train, y_train)




GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('data_processing',
                                        ColumnTransformer(transformers=[('poly_processing',
                                                                         Pipeline(steps=[('add_polynomials',
                                                                                          PolynomialFeatures(include_bias=False)),
                                                                                         ('standardize_poly',
                                                                                          StandardScaler())]),
                                                                         []),
                                                                        ('nonpoly_scaling',
                                                                         StandardScaler(),
                                                                         Index(['age', 'commuting_distance', 'education',
       'satisfaction_w...
       'years_since_last_promotion', 'years_with_current_supervisor'],
      dtype='object')),
                                                                        ('dummys',
                                                                         OneHotEncoder(drop='first'),
                                                                         Index(['frequency_of_travel', 'department', 'gender', 'position',
       'married_or_single'],
      dtype='object'))])),
                                       ('logreg',
                                        LogisticRegression(max_iter=500,
                                                           penalty=None))]),
             param_grid={'logreg__C': [0.01, 0.1, 1, 10, 100],
                         'logreg__penalty': ['l2']},
             scoring='accuracy')

In [13]:
accuracy=grid_search.best_score_
accuracy

0.69625

Though its clear that the possibility of the accuracy attained using the GridSearchCV model may only be able to represent the accuracy for that specific run and may not always represent the accuracy on the full set.

Logistic regression accuracy = 0.67 
However, GridSerachCV applies cross-validation to show accuracy over multiple data divisions, making this estimate more reliable. GridSearchCV's predictions were accurate ie  Accuracy of 0.69, ie greater than logistic regression, even when the top predictors weren't used.GridSearchCV's performed better than logistic regression.